In [ ]:
# Image data we use for training
# https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data?select=train.zip
# (could use !wget ...)

# For image files, Keras will automatically assign the name of the class (category) based on its parent folder name
# After downloading the data and moving them to the project directory, execute in the terminal:
!unzip train.zip
%mv train data
%cd data
%mkdir train val
%mkdir train/cat train/dog
%mkdir val/cat val/dog

In [ ]:
# Select randomly 250 images per class and place them into train and val folders
%ls | grep cat | sort -R | head -250 | xargs -I {} mv {} train/cat/
%ls | grep dog | sort -R | head -250 | xargs -I {} mv {} train/dog/
%ls | grep cat | sort -R | head -250 | xargs -I {} mv {} val/cat/
%ls | grep dog | sort -R | head -250 | xargs -I {} mv {} val/dog/

In [ ]:
!conda install -c conda-forge tensorflow --yes
!conda install -c conda-forge keras --yes
!conda install -c conda-forge pillow --yes
!conda install -c conda-forge matplotlib --yes

In [ ]:
# Import packages
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math
import os

In [ ]:
# Configurations
# We treat this problem as a multiclass problem, i.e. "cats vs dogs" (Number of classes 2)
TRAIN_DATA_DIR = 'data/train/'
VALIDATION_DATA_DIR = 'data/valid/'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

### Image Classification

- Input data are understood by DL algorithms as **tensors** (multidimensional array or list)

- Image data: 
    
    - Input batch of images 

    - 4D tensor:

<img src="Pictures/4_axis_tensor.png" style="width: 15%"/>


*https://www.tensorflow.org/guide/tensor*

- Images: 2 dimensions define Pixel values, 3rd RGB values, 4th number of images per batch

- In our case: $224*224*3*64$

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ?ImageDataGenerator
# Generates batches of image data with real-time data augmentation
# Loops over the data in batches
# ImageDataGenerator is a function provided by keras for augmenting the data while they are loaded
# preprocess_input: scale pixel values (0-255) of rgb channels to 0-1

In [ ]:
# Images are combined into batches, training single images is inefficient
# Shuffling to introduce more randomness
# Seed for reproducibility
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

In [ ]:
# Model definition
# include_top = FALSE: Throw away last few specific layers (fully connected layers)
def model_maker():
    base_model = MobileNet(include_top=False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False # freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)) # instantiate a Keras tensor
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation = 'relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [ ]:
model = model_maker()
# model.summary()

# Information on MobileNet and its arguments:
# https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/README.md
# ?MobileNet

In [ ]:
# Model training (i.e. model fitting)
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(0.001), 
              metrics=['acc'])
model.fit(
    train_generator, 
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE), 
    epochs=10, # epoch -> one full training step (network goes over entire dataset)
    validation_data=validation_generator, 
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE)) 
# Google colab can be used for running on GPU

In [ ]:
model.save('model_cd')

In [ ]:
# Testing the model on a sample image
# Load the model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('model_cd')

In [ ]:
# Load sample picture and check how the model performs
img_path = 'Pictures/sample_dog.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
display(img)
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image

# prediction > 0.5
# How does NN arrive at probabilities?
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)